## Importing packages

In [40]:
import numpy as np
import pandas as pd
import twitter
from twitter import Twitter
from twitter import OAuth

import json 
from pandas.io.json import json_normalize

ck = 'QvGBrXMEZhicazT2XzvK1usgY' #consumer key
cs = 'TyYiFfvYszrjtoikhn5dlMYWljuEPGQu3iUlPZmByZAgKwTgP2' #consumer key secret
at = '954075809018470400-2TRhibAbIA7OBhPUXuf6xE8NIMTWa17' 
ats = 'qbg2ibQ4QILAIYidy9kdANZ5cxSO2a9jCPhOIYmsGm1Wv'

oauth = OAuth(at,ats,ck,cs)

api = Twitter(auth=oauth)

## Extracting the tweets using API

In [42]:
df = pd.DataFrame()
mid = 0
for i in range(100):
    if i ==0:
        search_tw = api.search.tweets(q="Bitcoin", count = 100)
    else:
        search_tw = api.search.tweets(q="Bitcoin", count=100, max_id=mid)
    
    dftemp = json_normalize(search_tw,'statuses')
    mid = dftemp['id'].min()
    mid=mid-1
    df = df.append(dftemp,ignore_index=True)

In [43]:
df.shape

(9262, 30)

## Normalizing the fetched JSON data

In [44]:
tweet = df['text']
df_u = json_normalize(df['user'])
df_u.head()
df_s = df_u['screen_name']
df_s.head()
df['screenname']=df_u['screen_name']

## Importing NLP tool kit package Textblob

In [45]:
from textblob import TextBlob as tb

## Determining sentiment of tweets

In [46]:
pol = []
sub = []

for j in tweet:
    tx = tb(j)
    pol.append(tx.sentiment.polarity)
    sub.append(tx.sentiment.subjectivity)

In [47]:
df_pols = pd.DataFrame({"polarity":pol,"subjectivity":sub})
df['polarity']=df_pols['polarity']
df['subjectivity']=df_pols['subjectivity']

In [48]:
df_sup = pd.DataFrame()
df3 = pd.DataFrame()

In [49]:
df3 = df[['polarity','subjectivity','screenname']]
df3.head()

,polarity,subjectivity,screenname
0,0.136364,0.454545,Airdropnotecom
1,0.000000,0.000000,takigawa401
2,0.068182,0.227273,digiexchangeid
3,0.000000,0.000000,portal_bitcoin
4,0.000000,0.000000,CgAn_Doemela


## Classifying tweets into positive, negative, and neutral

In [58]:
negative = pd.DataFrame()
positive= pd.DataFrame()
neutral = pd.DataFrame()

negative = df3[df3['polarity']<=-0.4]
positive = df3[df3['polarity']>=0.4]
neutral = df3[(df3['polarity'] > -0.4) & (df3['polarity'] < 0.4)]

In [59]:
print((len(negative)/df.shape[0])*100 ,"percent of negative tweets ")

3.2822284603757286 percent of negative tweets 


In [60]:
negative.head()

,polarity,subjectivity,screenname
5,-1.000000,1.0,CoinWatcherBot
17,-1.000000,1.0,bitbrokersinc
21,-0.600000,0.9,johnmally49
29,-0.666667,1.0,IllumineCrypto2
37,-1.000000,1.0,newsinvesting


In [61]:
print((len(positive)/df.shape[0])*100 ,"percent of positive tweets ")

12.286763118117037 percent of positive tweets 


In [62]:
positive.head()

,polarity,subjectivity,screenname
7,0.500000,0.900,JaydeepPaul10
10,0.500000,1.000,ApolloCurrency
11,0.400000,0.600,btc_update
13,0.468182,0.625,goldcoinshinny
14,0.500000,0.750,lurlene77876014


In [63]:
print((len(neutral)/df.shape[0])*100 ,"percent of neutral tweets ")

84.43100842150724 percent of neutral tweets 


In [64]:
neutral.head()

,polarity,subjectivity,screenname
0,0.136364,0.454545,Airdropnotecom
1,0.000000,0.000000,takigawa401
2,0.068182,0.227273,digiexchangeid
3,0.000000,0.000000,portal_bitcoin
4,0.000000,0.000000,CgAn_Doemela
